# Análise de Jogos da Steam - Fase 2

Este notebook contém a análise solicitada na fase 2 do projeto, incluindo:
- Pré-processamento dos dados
- Respostas às perguntas da empresa
- Visualizações solicitadas
- Pergunta e visualização adicional

## 1. Importações e Configuração

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Adicionar o diretório raiz ao path para importar os módulos
sys.path.append(os.path.abspath('../..'))

# Importar funções criadas
from src.fase2.preprocessamento import preprocess_data, save_processed_data
from src.fase2.visualizacao import set_custom_style, plot_os_support

# Configurar o estilo personalizado para os gráficos
colors = set_custom_style()

# Configurações de exibição do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

ImportError: cannot import name 'set_custom_style' from 'src.fase2.visualizacao' (c:\Users\flavi\OneDrive\Documentos\GitHub\puc-trabalhos\analise-steam\src\fase2\visualizacao.py)

## 2. Carregamento e Pré-processamento dos Dados

In [ ]:
# Caminho para o arquivo de dados
data_path = '../../data/raw/steam_games.csv'

# Carregar os dados
df_raw = pd.read_csv(data_path)

# Exibir informações básicas sobre os dados
print(f"Quantidade de registros: {df_raw.shape[0]}")
print(f"Quantidade de colunas: {df_raw.shape[1]}")

# Visualizar as primeiras linhas
df_raw.head()

In [ ]:
# Pré-processar os dados
df = preprocess_data(df_raw)

# Salvar os dados processados
save_processed_data(df)

# Verificar as transformações
df.head()

## 3. Resposta às Perguntas da Empresa

### Pergunta 1: Os dez jogos mais bem avaliados pelo Metacritic

In [ ]:
# Filtrar para jogos com pontuação Metacritic válida
metacritic_df = df.dropna(subset=['metacritic_score'])

# Ordenar por pontuação (decrescente) e data de lançamento (crescente)
top_games = metacritic_df.sort_values(['metacritic_score', 'release_date'], 
                                      ascending=[False, True]).head(10)

# Selecionar apenas as colunas relevantes
top_games_display = top_games[['name', 'metacritic_score', 'release_date']]

print("Os dez jogos mais bem avaliados de acordo com o Metacritic:")
top_games_display

### Pergunta 2: Estatísticas para jogos de role-playing

In [ ]:
# Filtrar jogos de RPG
rpg_games = df[df['genres'].str.contains('RPG|role-playing', case=False, na=False)]

# Calcular estatísticas
rpg_stats = {
    'Número de jogos': len(rpg_games),
    'DLCs - Média': rpg_games['dlc_count'].mean(),
    'DLCs - Máximo': rpg_games['dlc_count'].max(),
    'Avaliações Positivas - Média': rpg_games['positive'].mean(),
    'Avaliações Positivas - Máximo': rpg_games['positive'].max(),
    'Avaliações Negativas - Média': rpg_games['negative'].mean(),
    'Avaliações Negativas - Máximo': rpg_games['negative'].max(),
    'Material de Demonstração - Média': rpg_games['demo_material'].mean(),
    'Material de Demonstração - Máximo': rpg_games['demo_material'].max()
}

# Criar DataFrame para exibição
rpg_stats_df = pd.DataFrame(rpg_stats.items(), columns=['Estatística', 'Valor'])
rpg_stats_df

### Pergunta 3: As cinco empresas que mais publicam jogos pagos

In [ ]:
# Filtrar jogos pagos
paid_games = df[df['is_paid'] == True]

# Contar número de jogos pagos por empresa
top_publishers = paid_games['publisher'].value_counts().reset_index()
top_publishers.columns = ['Publisher', 'Número de Jogos Pagos']
top_publishers = top_publishers.head(5)

# Calcular estatísticas de avaliações positivas
publisher_stats = []

for publisher in top_publishers['Publisher']:
    publisher_games = paid_games[paid_games['publisher'] == publisher]
    
    stats = {
        'Publisher': publisher,
        'Número de Jogos': len(publisher_games),
        'Avaliações Positivas - Média': publisher_games['positive'].mean(),
        'Avaliações Positivas - Mediana': publisher_games['positive'].median()
    }
    
    publisher_stats.append(stats)

# Criar DataFrame para exibição
publisher_stats_df = pd.DataFrame(publisher_stats)
publisher_stats_df

### Pergunta 4: Crescimento de jogos com suporte Linux entre 2018 e 2022

In [ ]:
# Filtrar jogos lançados entre 2018 e 2022
linux_trend = df[(df['release_year'] >= 2018) & (df['release_year'] <= 2022)]

# Agrupar por ano e contar jogos com suporte Linux
linux_by_year = linux_trend.groupby('release_year')['supports_linux'].agg(['sum', 'count']).reset_index()
linux_by_year['percentage'] = (linux_by_year['sum'] / linux_by_year['count']) * 100
linux_by_year.columns = ['Ano', 'Jogos com Linux', 'Total de Jogos', 'Percentual (%)']

# Visualizar os resultados
linux_by_year

## 4. Visualizações Solicitadas

### Gráfico 1: Percentual de jogos por sistema operacional

In [ ]:
# Gerar o gráfico usando a função criada
os_plot = plot_os_support(df)
plt.show()

### Gráfico 2: Jogos single-player do gênero Indie e estratégia por ano (2010-2020)

In [ ]:
# Filtrar jogos entre 2010 e 2020
time_range = df[(df['release_year'] >= 2010) & (df['release_year'] <= 2020)]

# Filtrar jogos single-player
single_player = time_range[time_range['categories'].str.contains('Single-player', case=False, na=False)]

# Filtrar jogos de cada gênero
indie_games = single_player[single_player['genres'].str.contains('Indie', case=False, na=False)]
strategy_games = single_player[single_player['genres'].str.contains('Strategy', case=False, na=False)]

# Contar por ano
indie_by_year = indie_games.groupby('release_year').size()
strategy_by_year = strategy_games.groupby('release_year').size()

# Criar figura
plt.figure(figsize=(12, 8))

# Plotar linhas
plt.plot(indie_by_year.index, indie_by_year.values, 'o-', linewidth=2, markersize=8, color=colors[0], label='Indie')
plt.plot(strategy_by_year.index, strategy_by_year.values, 's-', linewidth=2, markersize=8, color=colors[2], label='Strategy')

# Adicionar elementos visuais
plt.title('Número de Jogos Single-Player por Gênero (2010-2020)')
plt.xlabel('Ano de Lançamento')
plt.ylabel('Número de Jogos')
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)

# Configurar eixo x
plt.xticks(range(2010, 2021))

plt.tight_layout()
plt.show()

## 5. Pergunta e Visualização Adicional

### Pergunta Adicional: Qual a relação entre preço e avaliações positivas para os principais gêneros de jogos?

In [ ]:
# Identificar os 5 gêneros mais populares
genre_counts = {}
for genres in df['genres_list']:
    for genre in genres:
        if genre in genre_counts:
            genre_counts[genre] += 1
        else:
            genre_counts[genre] = 1

# Obter os 5 principais gêneros
top_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)[:5]
top_genre_names = [genre[0] for genre in top_genres]

# Filtrar jogos dos principais gêneros e com preço e avaliações válidos
valid_games = df.dropna(subset=['price', 'positive'])
valid_games = valid_games[valid_games['price'] <= 100]  # Remover outliers de preço

# Preparar dados para visualização
plot_data = []
for genre in top_genre_names:
    genre_games = valid_games[valid_games['genres'].str.contains(genre, case=False, na=False)]
    genre_data = genre_games[['name', 'price', 'positive', 'release_year']].copy()
    genre_data['genre'] = genre
    plot_data.append(genre_data)

# Combinar em um único DataFrame
plot_df = pd.concat(plot_data)

# Criar visualização
plt.figure(figsize=(14, 10))

for i, genre in enumerate(top_genre_names):
    genre_data = plot_df[plot_df['genre'] == genre]
    plt.scatter(genre_data['price'], 
                genre_data['positive'], 
                alpha=0.6, 
                s=80, 
                c=[colors[i % len(colors)]], 
                label=genre)

plt.xscale('log')  # Escala logarítmica para preço
plt.yscale('log')  # Escala logarítmica para avaliações

plt.title('Relação entre Preço e Avaliações Positivas por Gênero')
plt.xlabel('Preço (USD) - Escala Logarítmica')
plt.ylabel('Número de Avaliações Positivas - Escala Logarítmica')
plt.grid(True, alpha=0.3)
plt.legend(title='Gênero')

plt.tight_layout()
plt.show()

## 6. Conclusão

Neste notebook, realizamos uma análise completa do conjunto de dados de jogos da Steam, respondendo a diversas perguntas de negócio e criando visualizações informativas. As análises revelaram padrões interessantes sobre avaliações de jogos, gêneros populares, suporte a sistemas operacionais e tendências ao longo do tempo.